In [51]:
import numpy as np
import os, sys, json
import soundfile as sf
from converter import convert_wav

In [5]:
# load dictionary
with open('./data/va_dictionary.json') as f:
    va_dict = json.load(f)

dict_keys(['346_TheJungleBook_08_09BossEncounter', '042_CaptainTsubasaVol_II_SuperStriker_32_33CycloneLearning', '245_MoonCrystal_15_16Death', '162_HiryunoKenSpecial_FightingWars_00_01Title', '314_SummerCarnival_92_Recca_06_07ELM39Stage3Stage4', '135_GanbareGoemonGaiden2_TenkanoZaih__65_66AntarcticMap', '370_ValkyrienoBouken_TokinoKagiDensetsu_06_07BGMBUnderground', '301_Solstice_TheQuestfortheStaffofDemnos_01_02Introduction', '310_StarWars_TheEmpireStrikesBack_06_07Cantina', '312_Stinger_00_01Introduction', '255_NobunaganoYabou_BushouFuunroku_21_22Merchant', '228_MegaMan5_22_23Escape', '329_SwordMaster_14_15StaffRoll', '398_Ys_AncientYsVanishedOmen_13_14TowerofTheShadowofDeath3', '266_PoolofRadiance_13_14WildernessTheme', '142_Ghosts_nGoblins_10_112ndLapClear', '138_Gargoyle_sQuestII_01_02Opening', '248_MusashinoKen_TadaimaShugyouChuu_02_03FightStart', '178_Ironsword_Wizards_amp_WarriorsII_02_03GroundLevel', '396_YsIII_WanderersfromYs_17_18PrayerforLove', '291_ShadowoftheNinja_10_11En

In [14]:
def average_interval(fname):
    time, num = 0, 0
    with open(fname) as f:
        for line in f.readlines():
            words = line.split('_')
            if words[0] == 'WT' and int(words[1]) >= 200:
                time += int(words[1])
                num += 1
    return time//num

def get_arousal(fname):
    avg_interval = average_interval(fname)
    if avg_interval < 2500:
        return 1
    elif avg_interval < 5000:
        return 0
    else:
        return -1
    
def get_chroma(fname):
    chroma = [0]*12
    with open(fname) as f:
        for line in f.readlines():
            words = line.split('_')
            if len(words) == 3 and words[0] != 'NO':
                chroma[int(words[2]) % 12] += 1
    return chroma

Cmaj_key_prof = [6.35,2.23,3.48,2.33,4.38,4.09,2.52,5.19,2.39,3.66,2.29,2.88]
Cmin_key_prof = [6.33,2.68,3.52,5.38,2.60,3.53,2.54,4.75,3.98,2.69,3.34,3.17]
maj_prof = [] 
min_prof = [] 

for i in range(0,12): 
    maj_prof.append(np.roll(Cmaj_key_prof,i))
    min_prof.append(np.roll(Cmin_key_prof,i))

key_prof = maj_prof + min_prof
key_prof = np.transpose(key_prof)

def predict_key(fname):
    chroma = get_chroma(fname)
    cor = []
    for i in range(0,24):
        cor.append(np.corrcoef(get_chroma(fname),key_prof[:,i])[0,1])
    key_label = ['C','C#','D','Eb','E','F','F#','G','Ab','A','Bb','B','c','c#','d','eb','e','f','f#','g','ab','a','bb','b']
    return key_label[np.argmax(cor)]

def get_valence(fname):
    key_label = ['C','C#','D','Eb','E','F','F#','G','Ab','A','Bb','B','c','c#','d','eb','e','f','f#','g','ab','a','bb','b']
    return 1 if key_label.index(predict_key(fname)) < 12 else -1

In [19]:
def quantize_wait(wait_time):
    wait_time = min(wait_time, 100000)

    if wait_time > 10000:
        wait_time = 1000 * int(round(float(wait_time) / 1000) + 1e-4)
    elif wait_time > 1000:
        wait_time = 100 * int(round(float(wait_time) / 100) + 1e-4)
    elif wait_time > 100:
        wait_time = 10 * int(round(float(wait_time) / 10) + 1e-4)
    else:
        wait_time = round(wait_time)

    return wait_time

In [37]:
def convert(lines, key, v_from, a_from, v_to, a_to):
    key_label = ['C','C#','D','Eb','E','F','F#','G','Ab','A','Bb','B','c','c#','d','eb','e','f','f#','g','ab','a','bb','b']

    out = ''
    for line in lines:
        # arousal -> change of tempo
        words = line.split('_')
        if words[0] == 'WT':
            if a_to == 1:
                new_interval = 0.75 * int(words[1])
            elif a_to == -1:
                new_interval = 1.33 * int(words[1])
            else:
                new_interval = int(words[1])
            out += f'WT_{quantize_wait(new_interval)}\n'
        elif words[1] == 'NOTEON' and words[0] != 'NO':
            if v_from == -1 and v_to == 1:
                root_key = key_label.index(key) % 12
                if (int(words[2]) - root_key) % 12 in [3, 8, 10]:
                    out += f'{words[0]}_{words[1]}_{int(words[2])+1}\n'
                else:
                    out += line
            elif v_from == 1 and v_to == -1:
                root_key = key_label.index(key) % 12
                if (int(words[2]) - root_key) % 12 in [4, 9, 11]:
                    out += f'{words[0]}_{words[1]}_{int(words[2])-1}\n'
                else:
                    out += line
            else:
                out += line
        else:
            out += line
    return out

In [38]:
fn = './data/train/325_SuperMarioWorld_02_03Overworld.tx1.txt'
with open('./test.txt', 'w') as fw:
    with open(fn) as fr:
        fw.write(convert(fr.readlines(), predict_key(fn), get_valence(fn), get_arousal(fn), -1, -1))

In [46]:
v_s = (-1, 1)
a_s = (-1, 0, 1)

train_dir = './data/train/'
for v in v_s:
    for a in a_s:
        # create output directory
        out_dir = f'./output/target_v{v}_a{a}/'
        if not os.path.isdir(out_dir):
            os.makedirs(out_dir)
        # transform each files
        for file_name in os.listdir(train_dir):
            input_fn = train_dir + file_name
            with open(input_fn) as fr:
                with open(out_dir + file_name, 'w') as fw:
                    fw.write(convert(fr.readlines(), predict_key(input_fn), get_valence(input_fn), get_arousal(input_fn), v, a))
            print(f'{file_name} converted into {v} valence and {a} arousal')
                        

346_TheJungleBook_08_09BossEncounter.tx1.txt converted into -1 valence and -1 arousal
042_CaptainTsubasaVol_II_SuperStriker_32_33CycloneLearning.tx1.txt converted into -1 valence and -1 arousal
245_MoonCrystal_15_16Death.tx1.txt converted into -1 valence and -1 arousal
162_HiryunoKenSpecial_FightingWars_00_01Title.tx1.txt converted into -1 valence and -1 arousal
314_SummerCarnival_92_Recca_06_07ELM39Stage3Stage4.tx1.txt converted into -1 valence and -1 arousal
135_GanbareGoemonGaiden2_TenkanoZaih__65_66AntarcticMap.tx1.txt converted into -1 valence and -1 arousal
370_ValkyrienoBouken_TokinoKagiDensetsu_06_07BGMBUnderground.tx1.txt converted into -1 valence and -1 arousal
301_Solstice_TheQuestfortheStaffofDemnos_01_02Introduction.tx1.txt converted into -1 valence and -1 arousal
310_StarWars_TheEmpireStrikesBack_06_07Cantina.tx1.txt converted into -1 valence and -1 arousal
312_Stinger_00_01Introduction.tx1.txt converted into -1 valence and -1 arousal
255_NobunaganoYabou_BushouFuunroku_21

/home/hyeonseokchoi/anaconda3/envs/ml/lib/python3.9/site-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/hyeonseokchoi/anaconda3/envs/ml/lib/python3.9/site-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


268_PowerBlade2_10_11Stage5.tx1.txt converted into -1 valence and -1 arousal
335_TecmoCupSoccerGame_13_14OpponentGoal.tx1.txt converted into -1 valence and -1 arousal
233_MetalMax_13_14TankVodka.tx1.txt converted into -1 valence and -1 arousal
307_StarForce_05_06AreaTarget.tx1.txt converted into -1 valence and -1 arousal
084_DragonBuster_09_10GameOver.tx1.txt converted into -1 valence and -1 arousal
380_WaronWheels_09_10AtlantaArsenic.tx1.txt converted into -1 valence and -1 arousal
026_Batman_ReturnofTheJoker_09_10VSBoss.tx1.txt converted into -1 valence and -1 arousal
325_SuperMarioWorld_07_08Swimming.tx1.txt converted into -1 valence and -1 arousal
169_IceClimber_03_04GrabbedintoBigBird.tx1.txt converted into -1 valence and -1 arousal
089_DragonWarriorII_18_19LotterySuccess.tx1.txt converted into -1 valence and -1 arousal
123_Fire_nIce_01_02EnemyThemeProloguePt2.tx1.txt converted into -1 valence and -1 arousal
226_MegaMan3_12_13ProtoMan.tx1.txt converted into -1 valence and -1 arous

301_Solstice_TheQuestfortheStaffofDemnos_02_03MainTheme.tx1.txt converted into -1 valence and -1 arousal
246_Mother_23_24Fanfare2.tx1.txt converted into -1 valence and -1 arousal
178_Ironsword_Wizards_amp_WarriorsII_13_14EarthElemental.tx1.txt converted into -1 valence and -1 arousal
153_Gyromite_02_03Test.tx1.txt converted into -1 valence and -1 arousal
129_FreedomForce_17_18GameOver.tx1.txt converted into -1 valence and -1 arousal
156_HigemaruMakaijima_NanatsunoShimaDaibouken_09_10OokamiIsland.tx1.txt converted into -1 valence and -1 arousal
384_Wit_s_00_01Title.tx1.txt converted into -1 valence and -1 arousal
042_CaptainTsubasaVol_II_SuperStriker_37_38LastScene.tx1.txt converted into -1 valence and -1 arousal
179_JourneytoSilius_03_04Stage2.tx1.txt converted into -1 valence and -1 arousal
397_YsII_AncientYsVanishedTheFinalChapter_13_14InsideofTheIceWall.tx1.txt converted into -1 valence and -1 arousal
359_Town_amp_CountrySurfDesigns_Wood_amp_WaterRage_02_03Death.tx1.txt converted in

119_FinalFantasyIII_13_14ShrineofNept.tx1.txt converted into -1 valence and -1 arousal
342_TheFlintstones_TheSurpriseatDinosaurPeak__13_14GameOver.tx1.txt converted into -1 valence and -1 arousal
255_NobunaganoYabou_BushouFuunroku_12_13InvadingMouri.tx1.txt converted into -1 valence and -1 arousal
129_FreedomForce_06_07BonusGameStart.tx1.txt converted into -1 valence and -1 arousal
380_WaronWheels_03_04BostonBlueblood.tx1.txt converted into -1 valence and -1 arousal
091_DragonWarriorIV_00_01Overture.tx1.txt converted into -1 valence and -1 arousal
405_ZombieNation_09_10ZombienationRound21GrandCanyon.tx1.txt converted into -1 valence and -1 arousal
138_Gargoyle_sQuestII_02_03Mansion.tx1.txt converted into -1 valence and -1 arousal
030_BinaryLand_05_06NameEntry.tx1.txt converted into -1 valence and -1 arousal
129_FreedomForce_08_09BonusGame2.tx1.txt converted into -1 valence and -1 arousal
261_Parodius_05_06BossBGMDa.tx1.txt converted into -1 valence and -1 arousal
148_GradiusII_04_05For

...

174_ImageFight_04_05FlightovertheCityStage2.tx1.txt converted into 1 valence and 1 arousal
233_MetalMax_03_04ToanUnknownWilderness.tx1.txt converted into 1 valence and 1 arousal
274_R_C_Pro_Am_00_01TitleScreen.tx1.txt converted into 1 valence and 1 arousal
238_MikeTyson_sPunch_Out___05_06OpponentIsDown.tx1.txt converted into 1 valence and 1 arousal
330_TMNetwork_LiveinPowerBowl_11_12CarolsTheme.tx1.txt converted into 1 valence and 1 arousal
197_KujakuOuII_09_10Labyrinth.tx1.txt converted into 1 valence and 1 arousal
090_DragonWarriorIII_15_16Church.tx1.txt converted into 1 valence and 1 arousal
298_SolarJetman_HuntfortheGoldenWarpship_14_15CorsoQweroGameplay.tx1.txt converted into 1 valence and 1 arousal
226_MegaMan3_02_03StageSelect.tx1.txt converted into 1 valence and 1 arousal
316_SuperArabian_01_02StageClear.tx1.txt converted into 1 valence and 1 arousal
113_FantasyZoneII_TheTeardropofOpa_Opa_09_10Spardian.tx1.txt converted into 1 valence and 1 arousal
131_Galaga_09_10NameEntry2nd5

236_MickeyMousecapade_08_09TheCastle.tx1.txt converted into 1 valence and 1 arousal
267_Pooyan_08_09RockFallDanger2.tx1.txt converted into 1 valence and 1 arousal
167_Hudson_sAdventureIsland_02_03RoundClear.tx1.txt converted into 1 valence and 1 arousal
312_Stinger_01_02StageBegin1.tx1.txt converted into 1 valence and 1 arousal
232_MetalGear_00_01AerialInsertion.tx1.txt converted into 1 valence and 1 arousal
344_TheGooniesII_06_07PasswordTheme.tx1.txt converted into 1 valence and 1 arousal
165_HolyDiver_10_11PoisonMossContinentstage3.tx1.txt converted into 1 valence and 1 arousal
238_MikeTyson_sPunch_Out___03_04RoundStartGreatTigerBaldBullMikeTysonMrDream.tx1.txt converted into 1 valence and 1 arousal
372_Vs_BalloonFight_10_11Miss.tx1.txt converted into 1 valence and 1 arousal
191_Kirby_sAdventure_40_41NightmareWizard.tx1.txt converted into 1 valence and 1 arousal
203_LegacyofTheWizard_00_01OpeningTheme.tx1.txt converted into 1 valence and 1 arousal
115_FelixtheCat_03_04Level3.tx1.txt 

268_PowerBlade2_00_01Intro.tx1.txt converted into 1 valence and 1 arousal
077_DonkeyKongJr__17_18Stage3BGMVariation7.tx1.txt converted into 1 valence and 1 arousal
240_Milon_sSecretCastle_04_05DoorCloses.tx1.txt converted into 1 valence and 1 arousal
195_KonamiWaiWaiWorld_13_14KongStageBossBGM.tx1.txt converted into 1 valence and 1 arousal
298_SolarJetman_HuntfortheGoldenWarpship_31_32DestinySummons.tx1.txt converted into 1 valence and 1 arousal
153_Gyromite_03_04DirectTest.tx1.txt converted into 1 valence and 1 arousal
186_KidIcarus_06_07StageClear.tx1.txt converted into 1 valence and 1 arousal
042_CaptainTsubasaVol_II_SuperStriker_02_03FirstScene.tx1.txt converted into 1 valence and 1 arousal
127_FlyingWarriors_12_13Win.tx1.txt converted into 1 valence and 1 arousal
117_FinalFantasy_10_11Dungeon.tx1.txt converted into 1 valence and 1 arousal
045_Castlevania_10_11StageClear.tx1.txt converted into 1 valence and 1 arousal
250_NinjaGaiden_10_11TheCIA.tx1.txt converted into 1 valence and 

160_HiryunoKenIII_5NinnoRyuSenshi_06_07Boss1.tx1.txt converted into 1 valence and 1 arousal
327_SuperStarForce_JikuurekinoHimitsu_08_09Ending.tx1.txt converted into 1 valence and 1 arousal
162_HiryunoKenSpecial_FightingWars_12_13Unused2.tx1.txt converted into 1 valence and 1 arousal
210_LowGMan_TheLowGravityMan_14_15Pause.tx1.txt converted into 1 valence and 1 arousal
090_DragonWarriorIII_27_28MorninginAlefguard.tx1.txt converted into 1 valence and 1 arousal
378_WaiWaiWorld2_SOS__ParsleyJou_02_03RunRickle2.tx1.txt converted into 1 valence and 1 arousal
068_Dezaemon_05_06SampleGameBoss.tx1.txt converted into 1 valence and 1 arousal
136_GanbareGoemon_KarakuriDouchuu_05_06UnderpassSecretMaze.tx1.txt converted into 1 valence and 1 arousal
387_Wizardry_ProvingGroundsofTheMadOverlord_16_17Werdna.tx1.txt converted into 1 valence and 1 arousal
336_TenchioKurauII_ShokatsuKoumeiDen_14_15Cave1.tx1.txt converted into 1 valence and 1 arousal
236_MickeyMousecapade_03_04Boss.tx1.txt converted into 1 

053_ChoujinSentaiJetman_00_01Title.tx1.txt converted into 1 valence and 1 arousal
395_YoukaiDouchuuki_06_07Onigashima.tx1.txt converted into 1 valence and 1 arousal
060_CrisisForce_02_03Stage16.tx1.txt converted into 1 valence and 1 arousal
246_Mother_15_16Mystery.tx1.txt converted into 1 valence and 1 arousal
066_DestinyofanEmperor_05_06Encounter.tx1.txt converted into 1 valence and 1 arousal
122_FireEmblem_AnkokuRyutoHikarinoTsurugi_09_10Fight2CPSideAttack.tx1.txt converted into 1 valence and 1 arousal
088_DragonWarrior_24_25Curse.tx1.txt converted into 1 valence and 1 arousal
263_PhantomFighter_09_10Extermination.tx1.txt converted into 1 valence and 1 arousal
385_WizardryII_LlylgamynnoIsan_08_09MurmurChantPrayInvoke.tx1.txt converted into 1 valence and 1 arousal
036_BombermanII_15_16VSVictory.tx1.txt converted into 1 valence and 1 arousal
353_TheTowerofDruaga_05_06MusicWhenDruagaAppears.tx1.txt converted into 1 valence and 1 arousal
005_Abadox_TheDeadlyInnerWar_10_11Ending.tx1.txt c

In [62]:
fn = train_dir + '322_SuperMarioBros__02_03SwimmingAround.tx1.txt'
print(get_valence(fn), get_arousal(fn))

1 0


In [56]:
for file_name in os.listdir(train_dir):
    v, a = get_valence(train_dir + file_name), get_arousal(train_dir + file_name)
    if (v, a) == (-1, -1):
        print(file_name)

142_Ghosts_nGoblins_10_112ndLapClear.tx1.txt
138_Gargoyle_sQuestII_01_02Opening.tx1.txt
368_Ultima_QuestoftheAvatar_07_08Combat.tx1.txt
030_BinaryLand_00_01RoundStartMainBGM.tx1.txt
122_FireEmblem_AnkokuRyutoHikarinoTsurugi_20_21Story3Conversation.tx1.txt
089_DragonWarriorII_09_10Encounter.tx1.txt
087_DragonSpirit_TheNewLegend_17_18Ending.tx1.txt


/home/hyeonseokchoi/anaconda3/envs/ml/lib/python3.9/site-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/hyeonseokchoi/anaconda3/envs/ml/lib/python3.9/site-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


084_DragonBuster_09_10GameOver.tx1.txt
026_Batman_ReturnofTheJoker_09_10VSBoss.tx1.txt
064_DeepDungeonII_YuushinoMonshou_15_16GameOver.tx1.txt
118_FinalFantasyII_20_21GameOver.tx1.txt
191_Kirby_sAdventure_32_33QuickDraw.tx1.txt
215_Magician_03_04Wilderness.tx1.txt
147_Gradius_00_01Coin.tx1.txt
387_Wizardry_ProvingGroundsofTheMadOverlord_10_11Camp.tx1.txt
031_BioMiracleBokutteUpa_03_04TreasureChest.tx1.txt
307_StarForce_07_08Congratulation.tx1.txt
250_NinjaGaiden_06_07ASuddenDevelopment.tx1.txt
255_NobunaganoYabou_BushouFuunroku_19_20ShadowDeath.tx1.txt
284_RollingThunder_03_04MabusTheme.tx1.txt
091_DragonWarriorIV_41_42ViciousThing.tx1.txt
160_HiryunoKenIII_5NinnoRyuSenshi_20_21FudouAppearance.tx1.txt
064_DeepDungeonII_YuushinoMonshou_02_03Prayer.tx1.txt
137_GansoSaiyuuki_SuperMonkeyDaibouken_00_01TitleScreen.tx1.txt
215_Magician_17_18Credits.tx1.txt
090_DragonWarriorIII_04_05Adventure.tx1.txt
136_GanbareGoemon_KarakuriDouchuu_01_02Start.tx1.txt
119_FinalFantasyIII_28_29TheBoundlessOce

396_YsIII_WanderersfromYs_14_15SteelingTheWilltoFight.tx1.txt
138_Gargoyle_sQuestII_10_11Field.tx1.txt
000_10_YardFight_00_01GameStart.tx1.txt
167_Hudson_sAdventureIsland_10_11EvilSpirit.tx1.txt
310_StarWars_TheEmpireStrikesBack_07_08HappyOne.tx1.txt
160_HiryunoKenIII_5NinnoRyuSenshi_12_13WhenatthatTime.tx1.txt
345_TheGuardianLegend_10_11LevelUpCompileJingle.tx1.txt
174_ImageFight_17_18WhatsYourNameRanking.tx1.txt
253_Nobunaga_sAmbition_02_03OffenseandDefenseoftheWorldBattleScene.tx1.txt
142_Ghosts_nGoblins_07_08ThemeofSatan56StageBoss.tx1.txt
081_Dr_JekyllandMr_Hyde_05_06ElenaMcCowen.tx1.txt
010_AlienSyndrome_04_05KillerInstinct.tx1.txt
398_Ys_AncientYsVanishedOmen_00_01Feena.tx1.txt
254_Nobunaga_sAmbitionII_01_02OpeningRivalChiefsRisingtoAction.tx1.txt
091_DragonWarriorIV_35_36PadequiaSeed.tx1.txt
050_ChaosWorld_10_11ARestintheInn.tx1.txt
068_Dezaemon_02_03SampleGameStage1.tx1.txt
061_DarkLord_07_08ScenarioItem.tx1.txt
278_Renegade_12_13Ending.tx1.txt
250_NinjaGaiden_03_04MysteriousW

230_Meiky_JiinDababa_10_11Ending.tx1.txt
263_PhantomFighter_10_11House.tx1.txt
161_HiryunoKenII_DragonnoTsubasa_20_21TwirlRuins.tx1.txt
165_HolyDiver_00_01Title.tx1.txt
095_Excitebike_04_05StartBGM.tx1.txt
046_CastlevaniaIII_Dracula_sCurse_22_23Encounter.tx1.txt
328_SuperXevious_GAMPnoNazo_14_15BGMIArea2021.tx1.txt
127_FlyingWarriors_07_08SacredPlaceStore.tx1.txt
089_DragonWarriorII_03_04DistantJourney.tx1.txt
277_RainbowIslands_TheStoryofBubbleBobble2_12_13DariusIslandBGM.tx1.txt
253_Nobunaga_sAmbition_01_02PresentDayDreamsMainBGM.tx1.txt
345_TheGuardianLegend_11_12CorridorEntrance.tx1.txt
091_DragonWarriorIV_38_39FluteofUncovering.tx1.txt
091_DragonWarriorIV_18_19APleasantCasino.tx1.txt
269_ProYakyuu_FamilyStadium_88_03_04ChangeSide.tx1.txt
253_Nobunaga_sAmbition_04_05Defeat.tx1.txt
385_WizardryII_LlylgamynnoIsan_06_07AnnihilationRequiem.tx1.txt
091_DragonWarriorIV_14_15LevelUp.tx1.txt
233_MetalMax_18_19WanderersSong.tx1.txt
028_BattleCity_01_02GameOver.tx1.txt
185_KickMaster_07_08Ji

105_FamicomJump_HeroRetsuden_33_34GameOver.tx1.txt
398_Ys_AncientYsVanishedOmen_20_21RestinPeace.tx1.txt


In [63]:
fn = '322_SuperMarioBros__02_03SwimmingAround.tx1.txt'

sf.write('./MarioUnderwater.wav', convert_wav(train_dir + fn), samplerate=44100)
sf.write('./MarioUnderwater_dark_and_fast.wav', convert_wav('./output/target_v-1_a1/' + fn), samplerate=44100)

In [65]:
with open(train_dir + '322_SuperMarioBros__02_03SwimmingAround.tx1.txt') as f:
    print(len(f.readlines()))

948


In [64]:
fn = '298_SolarJetman_HuntfortheGoldenWarpship_12_13OmebruGameplay.tx1.txt'

sf.write('./Long.wav', convert_wav(train_dir + fn), samplerate=44100)

KeyboardInterrupt: 